## Ejercicios de pair programming 27 enero: Preparacion datos RL

Usando el mismo dataset que usatéis ayer, los objetivos de los ejercicios de hoy son:

-Estandarizar las variables numéricas de vuestro set de datos

-Codificar las variables categóricas. Recordad que tendréis que tener en cuenta si vuestras variables tienen orden o no.

-Chequear si vuestros datos están balanceados. En caso de que no lo estén utilizad algunas de las herramientas aprendidas en la lección para balancearlos.

-Guardad el dataframe con los cambios que habéis aplicado para utilizarlo en la siguiente lección.